In [1]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import datetime
import yfinance as yf
import time
import requests
import io

In [ ]:
# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL', 'TSLA', 'AMZN']

In [ ]:
# Get Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    print("current url is: " +url)
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36."}
    req = Request(url=url,headers=header) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass

current url is: https://finviz.com/quote.ashx?t=AAPL
current url is: https://finviz.com/quote.ashx?t=TSLA
current url is: https://finviz.com/quote.ashx?t=AMZN


Recent News Headlines for AAPL: 
What are the best iPad and tablet Black Friday deals? Here's what's expected in the 2022 UK sales ( Oct-14-22 01:00PM )
3 Reasons Apple Stock Is a Great Buy Today ( 09:41AM )
Is This Top Warren Buffett Dividend Stock a Buy Now? ( 08:15AM )


Recent News Headlines for TSLA: 
Tesla earnings are coming, but do record deliveries mask a demand problem? ( Oct-14-22 01:10PM )
US STOCKS-Wall St tumbles on profit drop from big banks, inflation worries ( 12:43PM )
Elon Musk says SpaceX cant afford to give Ukraine free Starlink access anymore ( 12:41PM )


Recent News Headlines for AMZN: 
What are the best iPad and tablet Black Friday deals? Here's what's expected in the 2022 UK sales ( Oct-14-22 01:00PM )
Why Peloton Interactive Stock Is Falling Once Again Today ( 12:42PM )
Amazon faces worker walkouts, s

In [ ]:
# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        print(x.get_text())
        text = x.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text ])

Oct-14-22 01:00PMWhat are the best iPad and tablet Black Friday deals? Here's what's expected in the 2022 UK sales The Telegraph
09:41AM3 Reasons Apple Stock Is a Great Buy Today Motley Fool
08:15AMIs This Top Warren Buffett Dividend Stock a Buy Now? Motley Fool
07:00AMMicrosoft Just Gave Meta Platforms a Big Vote of Confidence Motley Fool
05:52AM3 Best Buffett Stocks to Buy for the Long Haul Motley Fool
05:35AMReady for Long-Term Growth? You Can't Go Wrong With These 3 Stocks Motley Fool
05:30AMSnaps Stock Has Fallen Almost 80% This Year: What Went Wrong WSJ
03:35AMTSMC Jumps After Capex Cut and Better-Than-Expected Earnings Bloomberg
Oct-13-22 03:58PMBrazil court fines Apple, orders to sell iPhone with charger Reuters
03:54PMBrazil court fines Apple, orders to sell iPhone with charger Reuters
03:05PMBig promises  but will Apple, VinFast, Wolfspeed deliver in NC? Report shows incentives don't always mean jobs American City Business Journals
12:52PMApple teams up with Goldman Sachs on 

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ishaanpaul/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')

In [ ]:
# View Data 
news['Date'] = pd.to_datetime(news.Date).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers: 
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print ('\n')
    print (dataframe.head())
    
    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)
    
df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print ('\n')
print (df)



              Date     Time    neg    neu    pos  compound
Ticker                                                    
AAPL    2022-10-14  01:00PM  0.000  0.826  0.174    0.6369
AAPL    2022-10-14  09:41AM  0.193  0.533  0.273    0.2960
AAPL    2022-10-14  08:15AM  0.212  0.657  0.131   -0.2732
AAPL    2022-10-14  07:00AM  0.191  0.592  0.217    0.1027
AAPL    2022-10-14  05:52AM  0.170  0.585  0.246    0.3182


              Date     Time    neg    neu    pos  compound
Ticker                                                    
TSLA    2022-10-14  01:10PM  0.306  0.694  0.000   -0.6486
TSLA    2022-10-14  12:43PM  0.275  0.562  0.163   -0.2500
TSLA    2022-10-14  12:41PM  0.000  0.798  0.202    0.5106
TSLA    2022-10-14  12:33PM  0.122  0.878  0.000   -0.3321
TSLA    2022-10-14  11:57AM  0.000  0.769  0.231    0.5994


              Date     Time    neg    neu    pos  compound
Ticker                                                    
AMZN    2022-10-14  01:00PM  0.000  0.826  0.174  

In [ ]:
def getStockPrice(symbol):
    print("Getting stock price for stock $"+symbol)

getStockPrice("AAPL")

Getting stock price for stock $AAPL
